In [1]:

import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import tensorflow_probability.python.distributions as tfd
from statsmodels.tsa.stattools import adfuller

In [2]:
ZONES = ("22Bolo", "24Ama1", "26Fanh", "27Arr1", "29Sob2", "44Milagre", "48Vale_de", "60Maravil", "61MaraviII", "68Jogui")
HIST_NUM_YEARS = 4

In [3]:
# Hourly Wind generation data with multiple time series as columns 
# df = get_wind_prev_pivot(ZONES, HIST_NUM_YEARS)

In [4]:
#load df from pkl
df = pd.read_pickle('wind_generation_data.pkl')

In [5]:
df_29Sob = df['29Sob2']

# Check for stationarity

In [21]:
#Perform Dickey-Fuller test:
print ('Results of Dickey-Fuller Test:')
dftest = adfuller(df_29Sob.values, autolag='AIC')
dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations'])
for key, value in dftest[4].items():
    dfoutput['Critical Value (%s)'%key] = value
print (dfoutput)

Results of Dickey-Fuller Test:
Test Statistic           -1.824034e+01
p-value                   2.355766e-30
#Lags Used                4.800000e+01
Number of Observations    3.502400e+04
Critical Value (1%)      -3.430537e+00
Critical Value (5%)      -2.861623e+00
Critical Value (10%)     -2.566814e+00
dtype: float64


In [4]:
#df contains hourly wind generation data including historical data and forecasted for each time series. I want to split the data into historical and forecasted data based now time filtering the datetimeindex. getting df_hist and df_forecast
df_hist = df.loc[df.index < pd.Timestamp.now()]
df_forecast = df.loc[df.index >= pd.Timestamp.now()]

In [5]:
# Model parameters
num_timesteps, num_series = df_hist.shape
num_predictors = 5
horizon = 15 * 24  # 15 days ahead, hourly data

In [6]:

# Convert the data to a numpy array and change type to np.float32
observed_time_series = df_hist.values.astype(np.float32)

# Create some dummy predictors for illustration
predictors = tf.random.normal([num_timesteps, num_predictors])
# Example predictors
# predictors = tf.Variable(tf.random.normal([num_timesteps, num_predictors]), name='predictors')

In [7]:
trend = tfp.sts.LocalLinearTrend(observed_time_series=observed_time_series, name='trend')

#Daily seasonality for hourly data (24 hours)
seasonal = tfp.sts.Seasonal(num_seasons=24, observed_time_series=observed_time_series, name='seasonal')

#Weekly cycle for hourly data (24*7 hours)
cycle = tfp.sts.SmoothSeasonal(period=24*7, frequency_multipliers=[1., 2., 3.], observed_time_series=observed_time_series, name='cycle')  # Weekly cycle

regression = tfp.sts.LinearRegression(design_matrix=predictors, name='predictors')

In [12]:
# Define the model
sts_model  = tfp.sts.Sum([trend, seasonal, cycle, regression], observed_time_series=observed_time_series)

In [78]:
# def spike_and_slab_priors(num_predictors, num_series):
#     # Spike-and-slab prior for regression coefficients
#     slab_scale = tfp.util.TransformedVariable(1.0, tfb.Softplus())
#     spike_scale = tfp.util.TransformedVariable(0.1, tfb.Softplus())
# 
#     # Prepare an array of zeros of size num_predictors
#     loc_zeros = tf.zeros([num_predictors])
# 
#     # Prepare a list of mixture distributions
#     mixtures = [tfd.Normal(loc=loc_zeros, scale=spike_scale),
#                 tfd.Normal(loc=loc_zeros, scale=spike_scale),
#                 tfd.Normal(loc=loc_zeros, scale=spike_scale),
#                 tfd.Normal(loc=loc_zeros, scale=spike_scale),
#                 tfd.Normal(loc=loc_zeros, scale=spike_scale),
#                 tfd.Normal(loc=loc_zeros, scale=slab_scale)] * num_predictors
# 
#     spike_and_slab = tfd.MixtureSameFamily(
#         mixture_distribution=tfd.Categorical(logits=tf.zeros(num_predictors)),
#         components_distribution=tfd.Blockwise(mixtures)
#     )
#     priors = {
#         'regression_coefficients': spike_and_slab,
#         'observation_noise_scale': tfd.LogNormal(loc=0., scale=1.),
#     }
#     return priors
# 
# 
# # Define the spike-and-slab priors
# priors = spike_and_slab_priors(num_predictors, num_series)

In [74]:
# priors.values()
check_model_para = sts_model.parameters

In [59]:
# # Define the joint distribution
# def get_param_prior_values(parameters):
#     return [param.prior for param in parameters]
# 
# def sts_model_joint_distribution(observed_time_series, sts_model, num_timesteps, num_series):
#     param_vals = get_param_prior_values(sts_model.parameters)
# 
#     def sts_model_fn():
#         param_list = []
#         for param in param_vals:
#             param_list.append((yield param))
#         
#         # Create a state space model for the entire multivariate time series
#         state_space_model = sts_model.make_state_space_model(
#             num_timesteps=num_timesteps,
#             param_vals=param_list,
#             initial_state_prior=tfd.MultivariateNormalDiag(scale_diag=tf.ones([num_series]))
#         )
#         
#         # Log probability for multivariate time series
#         yield state_space_model.log_prob(observed_time_series)
# 
#     return tfd.JointDistributionCoroutineAutoBatched(sts_model_fn)


In [79]:
# # Define the joint distribution with priors
# def joint_distribution_fn():
#     param_list = []
#     for prior in priors.values():
#         param_list.append((yield prior))
#     
#     state_space_model = sts_model.make_state_space_model(
#         num_timesteps=num_timesteps,
#         param_vals=param_list,
#         initial_state_prior=tfd.MultivariateNormalDiag(scale_diag=tf.ones([num_series]))
#     )
#     yield state_space_model.log_prob(observed_time_series)
# 
# joint_dist = tfd.JointDistributionCoroutineAutoBatched(joint_distribution_fn)

In [89]:
# Define the joint distribution
def get_param_prior_values(parameters):
    return [param.prior for param in parameters]

def sts_model_joint_distribution(observed_time_series, sts_model, num_timesteps, num_series):
    param_vals = get_param_prior_values(sts_model.parameters)

    def sts_model_fn():
        param_list = []
        for param in param_vals:
            param_list.append((yield param))

        # Vectorized state space model
        def vectorized_model_fn(time_series):
            return sts_model.make_state_space_model(
                num_timesteps=num_timesteps,
                param_vals=param_list,
                initial_state_prior=tfd.MultivariateNormalDiag(scale_diag=tf.ones([sts_model.latent_size]))
            ).log_prob(time_series)

        log_probs = tf.vectorized_map(vectorized_model_fn, tf.transpose(observed_time_series))
        yield tf.reduce_sum(log_probs)

    return tfd.JointDistributionCoroutineAutoBatched(sts_model_fn)

In [45]:
# # Define the joint distribution
# def get_param_prior_values(parameters):
#     return [param.prior for param in parameters]
# 
# 
# def sts_model_joint_distribution(observed_time_series, sts_model, num_timesteps, num_series):
#     param_vals = get_param_prior_values(sts_model.parameters)
# 
#     def sts_model_fn():
#         param_list = []
#         for param in param_vals:
#             param_list.append((yield param))
#         
#         yield sts_model.make_state_space_model(
#             num_timesteps=num_timesteps,
#             param_vals=param_list,
#             initial_state_prior=tfd.MultivariateNormalDiag(scale_diag=tf.ones([num_series]))
#         ).log_prob(observed_time_series)
# 
#     return tfd.JointDistributionCoroutineAutoBatched(
#         model= sts_model_fn,
#         # sample_dtype=sts_model.dtype,
#         # use_vectorized_map=True
#     )

In [90]:
# Define the joint distribution and conditioning
joint_dist = sts_model_joint_distribution(observed_time_series, sts_model, num_timesteps, num_series)

In [91]:
# Pin the observed time series to the joint distribution
pinned_joint_dist = joint_dist.experimental_pin(observed_time_series=observed_time_series)

ValueError: x: required shape (34721, 1) but found (34721,)

In [11]:
# Inference using MCMC
num_results = 500
num_burnin_steps = 300

# Define the HMC transition kernel
@tf.function
def run_chain():
    hmc_kernel = tfp.mcmc.HamiltonianMonteCarlo(
        target_log_prob_fn=pinned_joint_dist.unnormalized_log_prob,
        step_size=0.1,
        num_leapfrog_steps=3
    )
    adaptive_hmc = tfp.mcmc.SimpleStepSizeAdaptation(
        inner_kernel=hmc_kernel,
        num_adaptation_steps=int(num_burnin_steps * 0.8),
        target_accept_prob=0.75
    )
    initial_chain_state = [tf.zeros_like(param.initial_value()) for param in sts_model.parameters]
    return tfp.mcmc.sample_chain(
        num_results=num_results,
        num_burnin_steps=num_burnin_steps,
        current_state=initial_chain_state,
        kernel=adaptive_hmc,
        trace_fn=lambda _, pkr: pkr.inner_results.is_accepted
    )

# Execute the MCMC chain
samples, is_accepted = run_chain()

In [12]:
# Forecast the future
def forecast(model, samples, num_steps_forecast):
    forecast_dist = tfp.sts.forecast(
        model, 
        observed_time_series=observed_time_series, 
        parameter_samples=samples, 
        num_steps_forecast=num_steps_forecast
    )
    
    return forecast_dist.sample(num_steps_forecast).numpy()

forecast_samples = forecast(sts_model, samples, horizon)